In [1]:
import numpy as np
import pandas as pd

weight = pd.read_csv("/home/calibretaliation/LinkedIn-Analysis/popularity_weight.csv")
weight

,actor1,actor2,weight
0,31,32,67.775
1,31,33,67.775
2,31,34,67.775
3,31,35,67.775
4,31,6751,67.775
...,...,...,...
751,3817649,3817687,67.775
752,3817649,3817690,67.775
753,3817652,3817687,67.775
754,3817652,3817690,67.775


In [2]:
actor_1 = weight['actor1'].unique().tolist()
actor_2 = weight['actor2'].unique().tolist()

actor_list = list(set(actor_1 + actor_2))
actor_list.insert(0, 0)
print(actor_list.index(3084))
#add ground node to list index
N = len(actor_list)
l = len(weight)
print("Number of actors: ", N)
print("Number of edges: ", l)
#Adjacent matrix P 
# P[i,j] = 0: no directed edge from i to j
# P[i,j] = 1: directed edge from i to j
#P[0,:]: ground node
P = np.zeros((N, N))
for i in range(1,N):
    P[0,i] = 1
    P[i,0] = 1
    if weight[weight['actor1'] == actor_list[i]] is not None:
        j_list = weight[weight['actor1'] == actor_list[i]]['actor2']
        for j in j_list:
            P[i, actor_list.index(j)] = 1   
            P[actor_list.index(j), i] = 1    
#Now divide adjacent matrix P by out degree of each node
D_out = P.sum(axis = 0)
D_in = P.sum(axis = 1)

for i in range(N):
    for j in range(N):

        P[j,i] = P[j,i]/D_out[j]

#Score matrix S
S = pd.DataFrame(data = {'actors': actor_list, 
                        'score': np.ones(N)})

S['score'][0] = 0
S


1
Number of actors:  99
Number of edges:  756


/tmp/ipykernel_33973/3533468052.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  S['score'][0] = 0


,actors,score
0,0,0.0
1,3084,1.0
2,3087,1.0
3,93210,1.0
4,1808925,1.0
...,...,...
94,53714,1.0
95,3106770,1.0
96,1517550,1.0
97,2072566,1.0


In [3]:
P_prime = P.transpose()


In [4]:
error = 100000
error_thres = 0.00001
old = S.copy()
iter = 0
P_prime = P.transpose()
while error > error_thres:
    iter += 1
    for i in range(N):
        S['score'][i] = sum(np.multiply(S['score'], P_prime[i]))
    error = sum(abs(S['score'][1:] - old['score'][1:]))/N
    print(error)
    old = S.copy()
print("ITERATION: ", iter)

0.11454357855428304
0.09236327748061383
0.06867392753351766
0.05122002424604302
0.039462174434058
0.03157453250956984
0.025803555424901068
0.022310798191939436
0.019571304862423117
0.017258927704445138
0.015289351226043154
0.01359777245635176
0.012133964260788891
0.01085866398371151
0.009740908031903818
0.008756052993695067
0.007884299316551746
0.007109584921817315
0.006418752309608442
0.0058009186201514506
0.005246996849341726
0.004749330035066342
0.004301410176206613
0.0038976609364742634
0.0035332685471595854
0.0032040492787708927
0.0029063447784687187
0.0026369387416420487
0.0023929900010302165
0.0021719783209604143
0.0019716600842214584
0.00179003173324697
0.0016252993335015638
0.0014758530079908756
0.001340245279347484
0.0012171725735112035
0.001105459304132677
0.0010040440825195725
0.0009119676939838539
0.0008283625551220729
0.0007524434233002581
0.0006834991735596649
0.000620885492368554
0.0005640183644216029
0.0005123682498069649
0.00046545486563451114
0.00042284249961865134
0

/tmp/ipykernel_33973/3440876870.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  S['score'][i] = sum(np.multiply(S['score'], P_prime[i]))


In [5]:
for i in range(1,N):
    S['score'][i] = sum(np.multiply(S['score'][1:], P_prime[i][1:]))


/tmp/ipykernel_33973/3369004321.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  S['score'][i] = sum(np.multiply(S['score'][1:], P_prime[i][1:]))


In [6]:
S.sort_values(by="score")

,actors,score
96,1517550,0.031689
29,1684099,0.063378
25,3814001,0.185470
56,206,0.185470
53,205,0.207373
...,...,...
7,33,1.831188
6,32,1.833443
5,31,1.835625
4,1808925,1.837737


In [11]:
out = S.sort_values(by="score", ascending = False).drop(0, axis = 0)
out['rank'] = 0
for i in range(len(out)):
    out['rank'][i] = i
out.drop('score', axis = 1, inplace = True)
out

/tmp/ipykernel_33973/932987476.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out['rank'][i] = i
/tmp/ipykernel_33973/932987476.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out['rank'][i] = i


,actors,rank
4,1808925,4
5,31,5
6,32,6
7,33,7
8,34,8
...,...,...
53,205,53
56,206,56
25,3814001,25
29,1684099,29


In [13]:
with open("LeaderRank.csv", "w") as file:
    out.to_csv(file)